In [ ]:
!pip install osmnx
!pip install geopy
!pip install folium

In [30]:
import osmnx as ox
import numpy as np
import math as math
import geopy
from geopy.distance import geodesic

In [31]:
# Charger les données OpenStreetMap pour Fès
graph = ox.graph_from_place("Fès, Maroc", network_type='drive')

In [32]:
# Récupérer la matrice d'adjacence du graphe.
adjacency = graph._adj

In [33]:
keys = adjacency.keys()

In [34]:
keys = list(keys)
keys.sort()

In [35]:
dic_k = {keys[0] : 0}
for i in range(len(keys)) :
  dic_k[keys[i]] = i


In [36]:
adj = {}
a=[]
for i, s in adjacency.items():  # Corrected colon placement

    a=[]
    for j in s.keys():
      a.append(dic_k[j])
    adj[dic_k[i]] = a

In [37]:
def adjacency_from_dict(adj_dict):
  """
  Convertit un dictionnaire d'attributs _adj en matrice adjacente.

  Args:
    adj_dict: Un dictionnaire d'attributs _adj.

  Returns:
    Une matrice adjacente.
  """

  # Créer une matrice vide.
  adjacency = np.zeros((len(adj_dict), len(adj_dict)))

  # Parcourir le dictionnaire _adj.
  for source, destination in adj_dict.items():
    for i in destination:
      # Mettre à jour la matrice adjacente.
      adjacency[source][i] = 1

  return adjacency

In [38]:
len(adj)

15917

In [39]:
L=adjacency_from_dict(adj)

In [40]:
nodes = graph._node
Graphe={}

for i in list(nodes.keys()):  # Corrected colon placement
    Graphe[dic_k[i]]=nodes[i]

In [41]:
def calculate_distance(coord1, coord2):
    # Calculate the distance
    distance = geodesic(coord1, coord2).meters
    return distance
def poids(u,v):
  global nodesGraph
  return calculate_distance((nodesGraph[v]['y'],nodesGraph[v]['x']), (nodesGraph[u]['y'],nodesGraph[u]['x']))

In [42]:
#34.037116, -5.002964
centre = {'y': 34.037116 , 'x':-5.002964}
newGraph = {}
for i in Graphe.items():
   if calculate_distance((i[1]['y'],i[1]['x']),(centre['y'],centre['x'])) < 500 :
    newGraph[i[0]]=i[1]

In [43]:
len(newGraph)

116

In [44]:
newGraph_list =list(newGraph.items())

In [45]:
newGraph_list.sort(key=lambda x: x[0])

In [46]:
# Créer une matrice de len(newGraph) x len(newGraph) remplie de zéros
matrice = np.zeros((len(newGraph), len(newGraph)))


In [47]:
for i in range(len(matrice)):
  for j in range(len(matrice)):
    matrice[i][j]= L[newGraph_list[i][0]][newGraph_list[j][0]]

In [48]:
nodesGraph={}
for i in range(len(newGraph_list)):
  nodesGraph[i] = newGraph_list[i][1]

In [49]:
len(matrice)

116

In [50]:
def poids(u,v):
  global nodesGraph
  return calculate_distance((nodesGraph[v]['y'],nodesGraph[v]['x']), (nodesGraph[u]['y'],nodesGraph[u]['x']))

In [51]:
def Bellman_Ford(s, graphe, dist, pred):
    # Nombre de sommets dans le graphe
    ns = len(graphe)

    # Initialisation des distances et des prédécesseurs
    for i in range(ns):
        dist.append(float('inf'))
        pred.append(None)

    # La distance du sommet initial à lui-même est 0
    dist[s] = 0

    # Relaxation des arêtes
    for i in range(1, ns):
        for node in range(ns):
            # Trouver les successeurs du noeud
            #successeurs = successeurfunction(node, graphe)
            successeurs = [v for v in range(ns) if matrice[node][v] == 1]
            for successeur in successeurs:
                # Calculer la distance temporaire
                tempdist = dist[node] + poids(node,successeur)
                # Mettre à jour la distance et le prédécesseur si nécessaire
                if dist[successeur] > tempdist:
                    dist[successeur] = tempdist
                    pred[successeur] = node

    # Vérification des cycles de poids négatif
    for node in range(ns):
        successeurs = [v for v in range(ns) if matrice[node][v] == 1]
        for successeur in successeurs:
            tempdist = dist[node] + poids(node,successeur)
            if dist[successeur] > tempdist:
                # Si on trouve un cycle de poids négatif, on retourne False
                return False

    # Si aucun cycle de poids négatif n'a été trouvé, on retourne True
    return True


In [52]:
def printShortPath(nodeofMygraphe,pred,s,d):
    path=[]
    path.append(nodeofMygraphe[d])
    m=pred[d]
    while m is not None:
        path.append(nodeofMygraphe[m])
        m=pred[m]
    path.reverse()
    listP =[]
    for i in range(len(path)):
      listP.append([path[i]['y'],path[i]['x']])
    return listP

In [53]:
s = 0
d=50
dist=[]
pred=[]
if Bellman_Ford(s,matrice,dist,pred):
    print('Le graphe ne contient pas de cercle négatif')
    chemin = printShortPath(nodesGraph,pred,s,d)
else:
    print('Le graphe contient un cercle négatif')

Le graphe ne contient pas de cercle négatif


In [54]:
_='''
des = {'y': 34.017837 , 'x':-5.052688}
min =10
indice = None
for i in range(len(nodesGraph)) :
  if min > poids(i,des) :
    min = poids(i,des)
    indice : i'''

In [55]:
#i= 29    34.0339453 , -5.0207892
#i= 11    34.0362908 , -5.0196482

In [56]:
import folium

# Créer une carte centrée sur un emplacement spécifique
m = folium.Map(location=chemin[0], zoom_start=50)

# Ajouter l'itinéraire à la carte
folium.Marker(location=chemin[0], icon=folium.Icon(color="green")).add_to(m)  # Départ
for i in range(1,len(chemin)-1):
  folium.Marker(location=chemin[i], icon=folium.Icon(color="blue")).add_to(m)  # Départ
folium.Marker(location=chemin[-1], icon=folium.Icon(color="red")).add_to(m)
folium.PolyLine(chemin, color='blue', weight=5, opacity=0.5).add_to(m)

# Afficher la carte
m